In [9]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import graphviz
from sklearn.linear_model import LassoCV

In [10]:
def load_data(telemetry_path: str, errors_path: str, failures_path: str, maint_path: str, machines_path: str, machine_id: int = 0) -> pd.DataFrame:
    def process_data(path: str, one_hot: bool = False) -> pd.DataFrame:
        df = pd.read_csv(path)
        df["datetime"] = pd.to_datetime(df["datetime"])

        if one_hot:
            df = pd.get_dummies(df,dtype=int)
            df = df.groupby(["datetime", "machineID"], as_index=False).max()
        
        df = df.set_index("datetime")
        df = df.sort_index()

        return df

    df_telemetry = process_data(telemetry_path)
    # print(df_telemetry.head())
    df_errors = process_data(errors_path, one_hot=True)
    df_failures = process_data(failures_path, one_hot=True)
    df_maint = process_data(maint_path, one_hot=True)
    # df_machine_info = pd.read_csv(machines_path, index_col="machineID")

    # df_merged = df_telemetry.reset_index().merge(right=df_machine_info, how="left", left_on="machineID", right_on="machineID", )
    df_merged = df_telemetry
    df_merged = df_merged.merge(right=df_errors, how="left", left_on=["datetime", "machineID"], right_on=["datetime", "machineID"])
    df_merged = df_merged.merge(right=df_failures, how="left", left_on=["datetime", "machineID"], right_on=["datetime", "machineID"])
    df_merged = df_merged.merge(right=df_maint, how="left", left_on=["datetime", "machineID"], right_on=["datetime", "machineID"])
    df_merged = df_merged.fillna(0)

    event_cols = [c for c in df_merged.columns if c.startswith(("errorID_", "failure_", "comp_"))]
    df_merged[event_cols] = df_merged[event_cols].astype(int)


    scaler = StandardScaler()
    for each in df_merged["machineID"].unique():
        mask = df_merged["machineID"] == each
        df_merged.loc[mask, ["volt", "rotate", "pressure", "vibration"]] = scaler.fit_transform(df_merged.loc[mask, ["volt", "rotate", "pressure", "vibration"]])

    rename_map = {"errorID_error1": "error_1", "errorID_error2": "error_2", "errorID_error3": "error_3", "errorID_error4": "error_4", "errorID_error5": "error_5",
                  "failure_comp1": "failure_c1", "failure_comp2": "failure_c2", "failure_comp3": "failure_c3", "failure_comp4": "failure_c4",
                  "comp_comp1": "maint_c1", "comp_comp2": "maint_c2", "comp_comp3": "maint_c3", "comp_comp4": "maint_c4"}
    df_merged = df_merged.rename(columns=rename_map)

    df_merged = df_merged.reset_index()
    df_merged = df_merged.sort_values(["machineID","datetime"]).set_index(["machineID","datetime"])

    if machine_id != 0:
        df_merged = df_merged[df_merged["machineID"] == machine_id]

    return df_merged

In [11]:
data = load_data(
    telemetry_path="./data/PdM_telemetry.csv",
    errors_path="./data/PdM_errors.csv",
    failures_path="./data/PdM_failures.csv",
    maint_path="./data/PdM_maint.csv",
    machines_path="./data/PdM_machines.csv"
)
data.head()

volt    rotate  pressure  vibration  \
machineID datetime                                                       
1         2015-01-01 06:00:00  0.351582 -0.532971  1.142729   0.812255   
          2015-01-01 07:00:00 -0.519455 -0.834698 -0.479554   0.510240   
          2015-01-01 08:00:00  0.010187  1.551347 -2.341734  -1.156200   
          2015-01-01 09:00:00 -0.546646 -1.918512  0.790105   0.096689   
          2015-01-01 10:00:00 -0.863543 -0.209869  1.033030  -2.633752   

                               error_1  error_2  error_3  error_4  error_5  \
machineID datetime                                                           
1         2015-01-01 06:00:00        0        0        0        0        0   
          2015-01-01 07:00:00        0        0        0        0        0   
          2015-01-01 08:00:00        0        0        0        0        0   
          2015-01-01 09:00:00        0        0        0        0        0   
          2015-01-01 10:00:00        0        0        0        0        0   

                               failure_c1  failure_c2  failure_c3  failure_c4  \
machineID datetime                                                              
1         2015-01-01 06:00:00           0           0           0           0   
          2015-01-01 07:00:00           0           0           0           0   
          2015-01-01 08:00:00           0           0           0           0   
          2015-01-01 09:00:00           0           0           0           0   
          2015-01-01 10:00:00           0           0           0           0   

                               maint_c1  maint_c2  maint_c3  maint_c4  
machineID datetime                                                     
1         2015-01-01 06:00:00         0         0         0         0  
          2015-01-01 07:00:00         0         0         0         0  
          2015-01-01 08:00:00         0         0         0         0  
          2015-01-01 09:00:00         0         0         0         0  
          2015-01-01 10:00:00         0         0         0         0

In [12]:
fail_df = pd.read_csv("./data/PdM_failures.csv", parse_dates=["datetime"])
error_df = pd.read_csv("./data/PdM_errors.csv", parse_dates=["datetime"])
maint_df = pd.read_csv("./data/PdM_maint.csv", parse_dates=["datetime"])

In [13]:
fail_df["datetime"] = pd.to_datetime(fail_df["datetime"])
error_df["datetime"] = pd.to_datetime(error_df["datetime"])
maint_df["datetime"] = pd.to_datetime(maint_df["datetime"])

In [14]:
error_df[error_df["datetime"].duplicated()]

,datetime,machineID,errorID
13,2015-04-19 06:00:00,1,error3
32,2015-10-16 06:00:00,1,error3
40,2015-03-18 06:00:00,2,error2
41,2015-03-18 06:00:00,2,error3
45,2015-04-17 06:00:00,2,error3
...,...,...,...
3911,2015-11-05 02:00:00,100,error3
3913,2015-11-12 01:00:00,100,error1
3915,2015-12-04 02:00:00,100,error1
3916,2015-12-08 06:00:00,100,error2


In [15]:
error_one_hot = pd.get_dummies(error_df,dtype=int)
error_one_hot[error_one_hot["datetime"].duplicated()]

,datetime,machineID,errorID_error1,errorID_error2,errorID_error3,errorID_error4,errorID_error5
13,2015-04-19 06:00:00,1,0,0,1,0,0
32,2015-10-16 06:00:00,1,0,0,1,0,0
40,2015-03-18 06:00:00,2,0,1,0,0,0
41,2015-03-18 06:00:00,2,0,0,1,0,0
45,2015-04-17 06:00:00,2,0,0,1,0,0
...,...,...,...,...,...,...,...
3911,2015-11-05 02:00:00,100,0,0,1,0,0
3913,2015-11-12 01:00:00,100,1,0,0,0,0
3915,2015-12-04 02:00:00,100,1,0,0,0,0
3916,2015-12-08 06:00:00,100,0,1,0,0,0


In [16]:
# 1) Should be exactly 8761 rows per machine in 2015 telemetry (if filtering a single machine)
print("rows:", len(data), "unique datetimes:", data.index.nunique())

# 2) No duplicated timestamps within a machine
assert not data.index.duplicated().any()


rows: 876100 unique datetimes: 876100


In [20]:
df = data.copy().reset_index()

def check_hourly(m):
    g = df[df["machineID"] == m]
    # duplicates
    assert not g.index.duplicated().any()
    # hourly grid
    full = g.asfreq("H")
    missing = full.isna().any(axis=1).sum()
    return len(g), missing, g.index.min(), g.index.max()

for m in df["machineID"].unique()[:5]:
    print(m, check_hourly(m))

1 (8761, 1, 0, 8760)
2 (8761, 1, 8761, 17521)
3 (8761, 1, 17522, 26282)
4 (8761, 1, 26283, 35043)
5 (8761, 1, 35044, 43804)


In [21]:
tele = pd.read_csv("./data/PdM_telemetry.csv", parse_dates=["datetime"])
tele_keys = set(zip(tele["machineID"], tele["datetime"]))

def match_rate(path, key_cols=("machineID","datetime")):
    ev = pd.read_csv(path, parse_dates=["datetime"])
    keys = list(zip(ev[key_cols[0]], ev[key_cols[1]]))
    return sum(k in tele_keys for k in keys) / len(keys)

print("errors match rate:", match_rate("./data/PdM_errors.csv"))
print("maint match rate:", match_rate("./data/PdM_maint.csv"))
print("fail match rate:", match_rate("./data/PdM_failures.csv"))

errors match rate: 1.0
maint match rate: 0.8782714546561169
fail match rate: 1.0


In [22]:
event_cols = [c for c in data.columns if c.startswith(("error", "maint_", "failure_"))]
by_machine_nonzero = df.groupby("machineID")[event_cols].sum()
print("min nonzero per col:", by_machine_nonzero.min().sort_values().head(10))

min nonzero per col: error_5       0
failure_c1    0
failure_c2    0
failure_c3    0
failure_c4    0
error_3       1
error_4       1
maint_c1      1
maint_c4      1
error_1       2
dtype: int64


In [23]:
overlap = 0
total_fail = 0
for k in [1,2,3,4]:
    f = df[f"failure_c{k}"] == 1
    m = df[f"maint_c{k}"] == 1
    overlap += (f & m).sum()
    total_fail += f.sum()
print("failures with same-hour maint:", overlap, "/", total_fail, "=", overlap/total_fail if total_fail else None)


failures with same-hour maint: 743 / 761 = 0.9763469119579501


In [24]:
tele = pd.read_csv("./data/PdM_telemetry.csv", parse_dates=["datetime"])
tmin, tmax = tele["datetime"].min(), tele["datetime"].max()

def unique_event_count(path, colname):
    df = pd.read_csv(path, parse_dates=["datetime"])
    df = df[(df["datetime"]>=tmin) & (df["datetime"]<=tmax)]
    return df.drop_duplicates(["machineID","datetime",colname])[colname].value_counts()

print("Errors raw:\n", unique_event_count("./data/PdM_errors.csv","errorID"))
print("Merged:\n", data.reset_index()[[c for c in data.columns if c.startswith("error_")]].sum())

Errors raw:
 error1    1010
error2     988
error3     838
error4     727
error5     356
Name: errorID, dtype: int64
Merged:
 error_1    1010
error_2     988
error_3     838
error_4     727
error_5     356
dtype: int64


In [25]:
df = data.reset_index()
for k in [1,2,3,4]:
    miss = df[(df[f"failure_c{k}"]==1) & (df[f"maint_c{k}"]==0)][["machineID","datetime"]]
    print(k, "fail without same-hour maint:", len(miss))


1 fail without same-hour maint: 9
2 fail without same-hour maint: 3
3 fail without same-hour maint: 3
4 fail without same-hour maint: 3


In [26]:
machines = pd.read_csv("./data/PdM_machines.csv")
assert machines["machineID"].nunique() == machines.shape[0]
assert set(data.reset_index()["machineID"].unique()).issubset(set(machines["machineID"].unique()))


In [27]:
tele = pd.read_csv("./data/PdM_telemetry.csv", parse_dates=["datetime"])
tmin, tmax = tele["datetime"].min(), tele["datetime"].max()

def unique_event_counts(path, colname):
    df = pd.read_csv(path, parse_dates=["datetime"])
    df = df[(df["datetime"]>=tmin) & (df["datetime"]<=tmax)]
    return df.drop_duplicates(["machineID","datetime",colname])[colname].value_counts()

print("Maint raw:\n", unique_event_counts("./data/PdM_maint.csv","comp"))
print("Merged maint:\n", data.reset_index()[[c for c in data.columns if c.startswith("maint_")]].sum())

print("Fail raw:\n", unique_event_counts("./data/PdM_failures.csv","failure"))
print("Merged fail:\n", data.reset_index()[[c for c in data.columns if c.startswith("failure_")]].sum())

Maint raw:
 comp2    763
comp4    711
comp3    708
comp1    704
Name: comp, dtype: int64
Merged maint:
 maint_c1    704
maint_c2    763
maint_c3    708
maint_c4    711
dtype: int64
Fail raw:
 comp2    259
comp1    192
comp4    179
comp3    131
Name: failure, dtype: int64
Merged fail:
 failure_c1    192
failure_c2    259
failure_c3    131
failure_c4    179
dtype: int64


In [28]:
tele = pd.read_csv("./data/PdM_telemetry.csv", parse_dates=["datetime"])
maint_raw = pd.read_csv("./data/PdM_maint.csv", parse_dates=["datetime"])
fail_raw  = pd.read_csv("./data/PdM_failures.csv", parse_dates=["datetime"])

# merged misses
df = data.reset_index()
for k in [1,2,3,4]:
    miss = df[(df[f"failure_c{k}"]==1) & (df[f"maint_c{k}"]==0)][["machineID","datetime"]]
    if len(miss) == 0:
        continue
    miss = miss.assign(comp=f"comp{k}")

    # look for a maint record within +/- 1 hour for same machine and component
    tmp = miss.merge(
        maint_raw[maint_raw["comp"]==f"comp{k}"][["machineID","datetime"]],
        on="machineID",
        how="left",
        suffixes=("_fail","_maint")
    )
    tmp["dt_hours"] = (tmp["datetime_maint"] - tmp["datetime_fail"]).dt.total_seconds()/3600
    near = tmp[tmp["dt_hours"].between(-1, 1)]
    print(f"comp{k}: misses={len(miss)}, near(+/-1h) matches rows={len(near)}")


comp1: misses=9, near(+/-1h) matches rows=0
comp2: misses=3, near(+/-1h) matches rows=0
comp3: misses=3, near(+/-1h) matches rows=0
comp4: misses=3, near(+/-1h) matches rows=0


In [29]:
machines = pd.read_csv("./data/PdM_machines.csv")
df = data.reset_index().merge(machines, on="machineID", how="left")

event_cols = [c for c in df.columns if c.startswith(("error_", "maint_", "failure_"))]
summary = df.groupby("model")[event_cols].sum()
print(summary)


        error_1  error_2  error_3  error_4  error_5  failure_c1  failure_c2  \
model                                                                         
model1      152      154      139      152       75          33          46   
model2      176      164      119      181       62          30          41   
model3      352      346      317      193      120          68          89   
model4      330      324      263      201       99          61          83   

        failure_c3  failure_c4  maint_c1  maint_c2  maint_c3  maint_c4  
model                                                                   
model1          68          42       107       117       112       122  
model2          63          34       120       129       110       126  
model3           0          64       241       297       258       236  
model4           0          39       236       220       228       227  
